In [1]:
import os
import site
import sqlite3
import sys

import logzero
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yaml
from logzero import logger
from tqdm import tqdm
from tqdm.notebook import tqdm
from yaml import dump, load, safe_load

In [2]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash_table import DataTable
from jupyter_dash import JupyterDash

In [3]:
sys.path.append("../../sql")
import queries

sys.path.append("../source")
import dbrd_eda
import dbrd_tools
import plot_tools
import ts_tools

In [4]:
log_path = "logs/"
log_file = "dashboard_eda.log"

logzero.logfile(log_path + log_file, maxBytes=1e6, backupCount=5, disableStderrLogger=True)
logger.info(f"{log_path}, {log_file}\n")

In [5]:
pd.set_option("precision", 4)

configs = None
try:
    with open("../configs/config.yml", "r") as config_in:
        configs = load(config_in, Loader=yaml.SafeLoader)
        logger.info(f"{configs}\n")
except:
    logger.error(f"config file open failure.")
    exit(1)

cfg_vars = configs["url_variables"]
logger.info(f"variables: {cfg_vars}\n")

years = configs["request_years"]
logger.info(f"years: {years}\n")

db_path = configs["file_paths"]["db_path"]

city = configs["location_info"]["city"]
state = configs["location_info"]["state"]
db_file = city + "_" + state + ".db"

db_table1 = configs["table_names"]["db_table1"]
db_table2 = configs["table_names"]["db_table2"]

logger.info(f"{db_path}, {db_file}")

nrows = configs["num_rows"][0]
logger.info(f"number of rows: {nrows}\n")

In [6]:
db_files = ts_tools.get_db_files(db_path)
logger.info(f"DB Path: {db_path}\n{db_files}\n")

In [7]:
def get_db_connection(db_filename=db_file):
    conn = sqlite3.connect(db_path + db_filename)
    logger.info(f"Connection made: {conn}")
    return conn

In [8]:
def get_db_zipcodes(conn):
    cursor = conn.cursor()
    cursor.execute(queries.select_distinct_zips)
    zipcodes = cursor.fetchall()
    zipcodes = [z[0] for z in zipcodes]
    logger.info(f"Distinct zip codes: {zipcodes}")
    return zipcodes

In [9]:
zipcode_index = 0

conn = get_db_connection(db_filename=db_file)
zipcodes = get_db_zipcodes(conn)

df = ts_tools.get_irr_data(conn, zipcodes[zipcode_index])

df_rsm = df.resample("M").mean()
# df_rsm.head(5)

In [10]:
fig1_units = ["Wh", "Wh", "Wh", "Wh", "°C", "%RH", "mm", "m/s", "stuff", "more_stuff"]
rows, cols = ts_tools.get_plots_layout(columns=4, column_names=fig1_units)
print(rows, cols)

2 4


In [11]:
14 % 4

2

In [12]:
df_desc = df_rsm.describe().transpose().round(decimals=3).reset_index(drop=False)

In [13]:
df_desc

,index,count,mean,std,min,25%,50%,75%,max
0,Clearsky_DNI,276.0,315.487,49.646,214.858,278.911,322.111,354.950,422.788
1,DNI,276.0,187.620,44.332,51.548,154.767,187.147,220.896,292.735
2,Clearsky_GHI,276.0,228.564,84.506,98.262,154.739,237.717,302.287,348.921
3,GHI,276.0,169.168,69.365,36.708,97.828,173.149,229.172,291.922
4,Temperature,276.0,9.983,8.989,-8.561,1.165,9.789,18.752,25.099
5,Relative_Humidity,276.0,81.489,5.048,62.013,78.729,82.265,85.129,91.174
6,Precipitable_Water,276.0,1.935,0.926,0.633,1.121,1.661,2.783,4.080
7,Wind_Speed,276.0,1.555,0.339,0.963,1.282,1.505,1.794,2.519


In [14]:
title1 = "Irradiance Raw Data"
fig1_units = ["Wh", "Wh", "Wh", "Wh", "°C", "%RH", "mm", "m/s"]
fig1 = plot_tools.plot_data(
    df_rsm,
    title=title1,
    zipcode=zipcodes[zipcode_index],
    columns=df_rsm.columns.tolist(),
    units=fig1_units,
)

In [15]:
title2 = "Data Distributions"
fig2 = plot_tools.plot_histograms(
    df_rsm,
    title=title2,
    zipcode=zipcodes[zipcode_index],
)

plot_histograms shape: 2, 4


In [16]:
title3 = "Meteorological Conditions"
columns = ["Temperature", "Relative_Humidity", "Precipitable_Water"]
fig3 = plot_tools.plot_multi_line(
    df_rsm,
    title=title3,
    columns=columns,
)

In [17]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE])

server = app.server

In [18]:
fig1_units = ["Wh", "Wh", "Wh", "Wh", "°C", "%RH", "mm", "m/s"]

app.layout = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(
                    dcc.Dropdown(
                        id="dd-db-selection",
                        options=[{"label": db, "value": db} for db in db_files],
                        value=db_files[0],
                        placeholder="Select a database",
                    ),
                    width={"size": 2, "offset": 1},
                ),
                dbc.Col(
                    dcc.Dropdown(
                        id="dd-zipcode-selection",
                        placeholder="Select a Zip Code",
                    ),
                    width={"size": 2},
                ),
            ],
            no_gutters=False,
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label(
                            "Data View",
                            style={"display": "inline-block", "textAlign": "center"},
                        ),
                        dcc.Graph(id="graph1", figure=fig1),
                    ],
                    width={"size": 5, "offset": 1},
                ),
                dbc.Col(
                    [
                        html.Label(
                            "Distribution View",
                            style={"display": "inline-block", "textAlign": "center"},
                        ),
                        dcc.Graph(id="graph2", figure=fig2),
                    ],
                    width={"size": 5},
                ),
            ],
            no_gutters=True,
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label(
                            "Meteorological View",
                            style={"display": "inline-block", "textAlign": "center"},
                        ),
                        dcc.Graph(id="graph3", figure=fig3),
                    ],
                    width={"size": 5, "offset": 1},
                ),
                dbc.Col(
                    [
                        html.Label(
                            "Desciptive Statistics",
                            style={
                                "display": "inline-block",
                                "textAlign": "center",
                            },
                        ),
                        DataTable(
                            data=df_desc.to_dict("records"),
                            columns=[{"id": col, "name": col} for col in df_desc.columns],
                            style_cell={
                                "padding": "5px",
                                "backgroundColor": "black",
                                "forgroundColor": "white",
                                "fontWeight": "bold",
                            },
                            style_header={
                                "padding": "5px",
                                "backgroundColor": "black",
                                "forgroundColor": "white",
                                "fontWeight": "bold",
                            },
                        ),
                    ],
                    width={"size": 5},
                ),
            ],
        ),
    ],
)

# end of layout
#
#
# begin callbacks


@app.callback(
    Output("dd-zipcode-selection", "options"),
    [
        Input("dd-db-selection", "value"),
    ],
)
def get_zipcodes(file_name):
    logger.info(f"get_zipcodes callback: {file_name}")

    conn = get_db_connection(db_filename=file_name)
    zipcodes = get_db_zipcodes(conn)
    conn.close()

    # return the list object to properly populate the dropdown!
    return [{"label": zipcode, "value": zipcode} for zipcode in zipcodes]


@app.callback(
    Output("dd-zipcode-selection", "value"),
    [
        Input("dd-zipcode-selection", "options"),
    ],
)
def set_zipcode_value(options):
    return options[0]["value"]


@app.callback(
    Output("ctx_output", "children"),
    Input("dd-db-selection", "value"),
    Input("dd-zipcode-selection", "value"),
)
def get_data_request(file_name, zipcode):
    ctx = dash.callback_context

    ctx_msg = json.dumps(
        {
            "states": ctx.states,
            "triggered": ctx.triggered,
            "inputs": ctx.inputs,
        },
        indent=2,
    )

    logger.info(f"get_data_request file name: {file_name}")
    logger.info(f"get_data_request zipcode: {zipcode}")
    logger.info(f"get_data_request Context message\n{ctx_msg}")

    #     conn = get_db_connection(db_filename=file_name)
    #     df = ts_tools.get_irr_data(conn, zipcode)
    #     display(df.head(5))
    #     conn.close()
    return f"{ctx_msg}"  # html.P(ctx_msg)

In [19]:
if __name__ == "__main__":
    app.run_server(mode="external", host="192.168.64.164", port="8080", use_reloader=False, debug=False)

 * Running on http://192.168.64.164:8080/ (Press CTRL+C to quit)
192.168.64.164 - - [08/Aug/2021 13:43:41] "GET /_alive_da1e5d3e-2798-4938-8988-ab3a5b8d45d6 HTTP/1.1" 200 -


Dash app running on http://192.168.64.164:8080/


192.168.64.64 - - [08/Aug/2021 13:43:43] "GET / HTTP/1.1" 200 -
192.168.64.64 - - [08/Aug/2021 13:43:43] "GET /assets/reset.css?m=1628374767.0 HTTP/1.1" 304 -
192.168.64.64 - - [08/Aug/2021 13:43:44] "GET /_favicon.ico?v=1.21.0 HTTP/1.1" 200 -
192.168.64.64 - - [08/Aug/2021 13:43:44] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.64.64 - - [08/Aug/2021 13:43:44] "GET /_dash-layout HTTP/1.1" 200 -
192.168.64.64 - - [08/Aug/2021 13:43:44] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
192.168.64.64 - - [08/Aug/2021 13:43:44] "GET /_dash-component-suites/dash_core_components/async-dropdown.js HTTP/1.1" 200 -
192.168.64.64 - - [08/Aug/2021 13:43:44] "GET /_dash-component-suites/dash_table/async-highlight.js HTTP/1.1" 200 -
192.168.64.64 - - [08/Aug/2021 13:43:44] "GET /_dash-component-suites/dash_table/async-table.js HTTP/1.1" 200 -
192.168.64.64 - - [08/Aug/2021 13:43:44] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 